In [58]:
import obspy
import numpy as np
import math
import cmath
import serveobspy
from obspy import UTCDateTime, read, Trace, Stream
import dragrace
import SeismogramTasks as sTasks

import matplotlib
%matplotlib inline

SPRING 2019

In [60]:
#load all hnz data points and subtract 4096 from the values
zvals = []
yvals = []
xvals = []
gravity = 4096
with open('timestamp_Sp_ALL.txt', 'r') as file_in:
#with open('timestamp_Fa_ALL.txt', 'r') as file_in:
    for line in file_in:
        try:
            line = line.rstrip()
            start = line
            for sta in ['FL','NL','NR','FR']:
                filename = 'SP_' + str(sta) + "_"  + str(line)
                fileout = 'SPr_' + str(sta) + '_' + str(line) + '_'
                stx = obspy.read("./springClean/Original/" + str(filename) + "_HNX.mseed")
                sty = obspy.read("./springClean/Original/" + str(filename) + "_HNY.mseed")
                stz = obspy.read("./springClean/Original/" + str(filename) + "_HNZ.mseed")
                #stx = obspy.read("./fallClean/Original/" + str(filename) + "_HNX.mseed")
                #sty = obspy.read("./fallClean/Original/" + str(filename) + "_HNY.mseed")
                #stz = obspy.read("./fallClean/Original/" + str(filename) + "_HNZ.mseed")
                #print("orig {}  {}  {}  {}".format(sta, stx[0][0], sty[0][0], stz[0][0]))
                #print(sta)
                if sta == 'FL':
                    theta = 70.0
                    alpha = 0.0
                elif sta == 'NL':
                    theta = 90.0
                    alpha = 0.0
                elif sta == 'NR':
                    theta = -90.0
                    alpha = 180.0
                elif sta == 'FR':
                    theta = -110.0
                    alpha = 180.0
                else:
                    print("wrong station value")
                #this function uses a list
                #[xprime,yprime,zprime] = sTasks.Local2Global_Pseudo3D_Rotation(stx[idx].data,sty[idx].data,stz[idx].data,theta,alpha)

                [xprime,yprime,zprime] = sTasks.Rotate_3D_TimeSeries(stx[0].data,sty[0].data,stz[0].data,theta,alpha)
                #print("rot raw {}  {}  {}  {}".format(sta, xprime[0], yprime[0], zprime[0]))
                stx[0].data = np.array(xprime)
                sty[0].data = np.array(yprime)
                stz[0].data = np.array(zprime)
                stz[0].data = stz[0].data - gravity
                stx.write("./springClean/Corrected/" + str(fileout) + "HNX.mseed", format='MSEED') 
                sty.write("./springClean/Corrected/" + str(fileout) + "HNY.mseed", format='MSEED') 
                stz.write("./springClean/Corrected/"  + str(fileout) + "HNZ.mseed", format='MSEED') 
                #stx.write("./fallClean/Corrected/" + str(fileout) + "HNX.mseed", format='MSEED') 
                #sty.write("./fallClean/Corrected/" + str(fileout) + "HNY.mseed", format='MSEED') 
                #stz.write("./fallClean/Corrected/"  + str(fileout) + "HNZ.mseed", format='MSEED')
                #print("rot saved {}  {}  {}  {}".format(sta, stx[0][0], sty[0][0], stz[0][0]))
                zvals.append(stz[0][0])
                yvals.append(sty[0][0])
                xvals.append(stx[0][0])
                    
        except Exception as e:
            print(e, "error")
        #break
    
#print(xvals)
#print(yvals)
#print(zvals)
#print("xxxxxxxxxxxxx")

[Errno 2] No such file or directory: './springClean/Original/SP_NR_2019-04-27T19:45:35Z_HNX.mseed' error
[Errno 2] No such file or directory: './springClean/Original/SP_NR_2019-04-27T19:48:23Z_HNX.mseed' error
[Errno 2] No such file or directory: './springClean/Original/SP_FL_2019-04-27T19:01:04Z_HNX.mseed' error
[Errno 2] No such file or directory: './springClean/Original/SP_FL_2019-04-27T19:04:22Z_HNX.mseed' error


In [61]:
#check the components
import statistics as stat
zvals = []
yvals = []
xvals = []
with open('timestamp_Sp_sunday.txt', 'r') as file_in:
    for line in file_in:
        try:
            line = line.rstrip()
            start = line
            for sta in ['FL','NL','NR','FR']:
                fileout = 'SPr_' + str(sta) + '_' + str(line) + '_'
                stzr = obspy.read("./springClean/Corrected/"  + str(fileout) + "HNZ.mseed")
                styr = obspy.read("./springClean/Corrected/"  + str(fileout) + "HNY.mseed")
                stxr = obspy.read("./springClean/Corrected/"  + str(fileout) + "HNX.mseed")
                print("rot saved {}  {}  {}  {}".format(sta, stxr[0][0], styr[0][0], stzr[0][0]))
                #print('stzr',stzr) # Check that it gets only HNZ (confirmed)
                zvals.append(stzr[0][0])
                yvals.append(styr[0][0])
                xvals.append(stxr[0][0])

        except Exception as e:
            print(e, "error")
        break
print(zvals)
print(yvals)
print(xvals)
zmean = stat.mean(zvals)
print(zmean,"zmean")
print(zmean/4096,"g zmean")


rot saved FL  32.874170728629906  40.0  -102.40008402207332
rot saved NL  -8.999999999999753  12.0  -54.0
rot saved NR  7.0000000000002505  -79.0000000000005  -3.0
rot saved FR  -33.55821101528045  -35.00000000000049  -100.52069878050088
[-102.40008402207332, -54.0, -3.0, -100.52069878050088]
[40.0, 12.0, -79.000000000000497, -35.00000000000049]
[32.874170728629906, -8.9999999999997531, 7.0000000000002505, -33.558211015280449]
-64.9801957006 zmean
-0.015864305591 g zmean


FALL 2019

In [65]:
#load all hnz data points and subtract 4096 from the values
zvals = []
yvals = []
xvals = []
gravity = 4096
with open('timestamp_Fa_ALL_J.txt', 'r') as file_in:
    for line in file_in:
        try:
            line = line.rstrip()
            start = line
            for sta in ['FL','NL','FL0','FL60','FL330','FL660','FL1K']:
                filename = 'FA_' + str(sta) + "_"  + str(line)
                fileout = 'FAr_' + str(sta) + '_' + str(line) + '_'
                stx = obspy.read("./fallClean/Original/" + str(filename) + "_HNX.mseed")
                sty = obspy.read("./fallClean/Original/" + str(filename) + "_HNY.mseed")
                stz = obspy.read("./fallClean/Original/" + str(filename) + "_HNZ.mseed")
                #print("orig {}  {}  {}  {}".format(sta, stx[0][0], sty[0][0], stz[0][0]))
                #print(sta)
                if (sta == 'FL') or (sta == 'FL0') or (sta == 'FL60') or (sta == 'FL330') or (sta == 'FL660') or (sta == 'FL1K'):
                    theta = 70.0
                    alpha = 0.0
                    print(sta, theta)
                elif sta == 'NL':
                    theta = 90.0
                    alpha = 0.0
                else:
                    print("wrong station value")
                #this function uses a list
                #[xprime,yprime,zprime] = sTasks.Local2Global_Pseudo3D_Rotation(stx[idx].data,sty[idx].data,stz[idx].data,theta,alpha)

                [xprime,yprime,zprime] = sTasks.Rotate_3D_TimeSeries(stx[0].data,sty[0].data,stz[0].data,theta,alpha)
                #print("rot raw {}  {}  {}  {}".format(sta, xprime[0], yprime[0], zprime[0]))
                stx[0].data = np.array(xprime)
                sty[0].data = np.array(yprime)
                stz[0].data = np.array(zprime)
                stz[0].data = stz[0].data - gravity
                stx.write("./fallClean/Corrected/" + str(fileout) + "HNX.mseed", format='MSEED') 
                sty.write("./fallClean/Corrected/" + str(fileout) + "HNY.mseed", format='MSEED') 
                stz.write("./fallClean/Corrected/"  + str(fileout) + "HNZ.mseed", format='MSEED') 
                #print("rot saved {}  {}  {}  {}".format(sta, stx[0][0], sty[0][0], stz[0][0]))
                zvals.append(stz[0][0])
                yvals.append(sty[0][0])
                xvals.append(stx[0][0])
                    
        except Exception as e:
            print(e, "error")
        #break
#print("x_____________",xvals)
#print("y_____________",yvals)
#print("z______________",zvals)
#print("xxxxxxxxxxxxx")

FL 70.0
FL0 70.0
FL60 70.0
FL330 70.0
FL660 70.0
FL1K 70.0
[Errno 2] No such file or directory: './fallClean/Original/FA_FL_2019-10-12T00:03:44Z_HNX.mseed' error
[Errno 2] No such file or directory: './fallClean/Original/FA_FL_2019-10-12T00:19:35Z_HNX.mseed' error
[Errno 2] No such file or directory: './fallClean/Original/FA_FL_2019-10-12T00:22:51Z_HNX.mseed' error
[Errno 2] No such file or directory: './fallClean/Original/FA_FL_2019-10-12T00:26:20Z_HNX.mseed' error
[Errno 2] No such file or directory: './fallClean/Original/FA_FL_2019-10-12T00:29:45Z_HNX.mseed' error
FL 70.0
[Errno 2] No such file or directory: './fallClean/Original/FA_NL_2019-10-12T18:36:00Z_HNX.mseed' error
[Errno 2] No such file or directory: './fallClean/Original/FA_FL_2019-10-12T18:44:03Z_HNX.mseed' error
[Errno 2] No such file or directory: './fallClean/Original/FA_FL_2019-10-12T18:47:00Z_HNX.mseed' error
[Errno 2] No such file or directory: './fallClean/Original/FA_FL_2019-10-12T18:49:51Z_HNX.mseed' error
[Errno